In [284]:
import requests

def retrieve_data(url):
    """
    Retrieve data from a URL.
    """
    try:
        response = requests.get(url, headers={'Content-Type': 'application/json'})
        if response.ok:
            return response
    except:
        print("Error: RemoteNumberDataSource.get()")
        return None

In [285]:
import json

response = retrieve_data("https://bet.hkjc.com/contentserver/jcbw/cmc/last30draw.json")
json_data = json.loads(response.content)
print(json_data[0])

{'id': '22/042', 'date': '05/07/2022', 'no': '9+30+31+32+37+40', 'sno': '25', 'sbcode': '', 'sbnameE': '', 'sbnameC': '', 'inv': '34,534,407', 'p1': '0', 'p1u': '0.0', 'p2': '0', 'p2u': '0.0', 'p3': '123,910', 'p3u': '37.5', 'p4': '9,600', 'p4u': '122.0', 'p5': '640', 'p5u': '2,509.9', 'p6': '320', 'p6u': '3,720.3', 'p7': '40', 'p7u': '47,854.1'}


In [286]:
numbers_data = [i["no"] for i in json_data]
numbers_data = [x.split("+") for x in numbers_data]
print(numbers_data[:2])

[['9', '30', '31', '32', '37', '40'], ['19', '21', '23', '26', '37', '39']]


In [287]:
import collections
import numpy as np

def counter_numbers(numbers):
    """
    Count the number of times each number appears in the last 30 draws.
    """
    numbers = np.array(numbers)
    counter = collections.Counter(numbers.flatten())
    return counter

counter = counter_numbers(numbers_data)
print(counter)

Counter({'30': 8, '37': 7, '46': 7, '40': 6, '21': 6, '48': 6, '8': 6, '9': 5, '26': 5, '39': 5, '1': 5, '29': 5, '15': 5, '34': 5, '38': 5, '11': 5, '35': 5, '32': 4, '19': 4, '20': 4, '47': 4, '17': 4, '7': 4, '12': 4, '28': 4, '23': 3, '2': 3, '27': 3, '16': 3, '41': 3, '42': 3, '36': 3, '31': 2, '13': 2, '24': 2, '33': 2, '44': 2, '43': 2, '25': 2, '5': 2, '49': 2, '45': 2, '4': 2, '10': 2, '22': 2, '18': 2, '14': 1, '3': 1, '6': 1})


In [288]:
def retrieve_most_number(counter):
    """
    Get all the most number.
    """
    numbers = []
    most_numbers = {}
    for k in counter.keys():
        index = counter[k]
        numbers = most_numbers[index]  if index in most_numbers else []
        numbers.append(k)
        most_numbers[index] = numbers

    most_numbers = sorted(most_numbers.items(), key=lambda x: x[0], reverse=True)
    return most_numbers

def get_all_most_number(counter):
    """
    Show the most number.
    """
    # print(f'===============Most of numbers==============')
    numbers = {}
    most_numbers = retrieve_most_number(counter)
    for v in most_numbers:
        key = v[0]
        values = sorted(v[1], key=lambda x: int(x))
        # print(f'p{key}', values)
        numbers.update({f'p{key}': values})
    return numbers

def get_particular_number(counter, by, to):
    """
    Show the particular number.
    """
    # print(f'===============[{by},{to}]==============')
    numbers_list = []
    numbers = {}
    most_numbers = retrieve_most_number(counter)
    for v in most_numbers:
        numbers_list.clear()
        key = v[0]
        values = sorted(v[1], key=lambda x: int(x))
        for i in values:
            if int(i) in range(by, to):
                numbers_list.append(i)
        if len(numbers_list) > 0: numbers.update({f'p{key}': numbers_list})
    return numbers
        
print(numbers_data[19])
all_most_numbers = get_all_most_number(counter)
get_particular_number(counter, 1, 10)
get_particular_number(counter, 10, 20)
get_particular_number(counter, 20, 30)
get_particular_number(counter, 30, 40)
get_particular_number(counter, 40, 50)

print(all_most_numbers)

['9', '23', '30', '32', '34', '40']
{'p8': ['30'], 'p7': ['37', '46'], 'p6': ['8', '21', '40', '48'], 'p5': ['1', '9', '11', '15', '26', '29', '34', '35', '38', '39'], 'p4': ['7', '12', '17', '19', '20', '28', '32', '47'], 'p3': ['2', '16', '23', '27', '36', '41', '42'], 'p2': ['4', '5', '10', '13', '18', '22', '24', '25', '31', '33', '43', '44', '45', '49'], 'p1': ['3', '6', '14']}


In [289]:
import math

def search_numbers(numbers, by, to):
    """
    Search for a number in a list of numbers.
    """
    slots = []
    for i in numbers:
        for k in i:
            if int(k)in range(by, to):
                slots.append(k)
    return slots

def show_percentage(numbers, by, to):
    """
    Show the percentage of numbers in a list of numbers.
    """
    slots = search_numbers(numbers, by, to)
    percent = math.ceil((len(slots) / (len(numbers) * 6)) * 100)
    print("{} ~ {} : {}%".format(by, to, percent))

show_percentage(numbers_data, 1, 10)
show_percentage(numbers_data, 10, 20)
show_percentage(numbers_data, 20, 30)
show_percentage(numbers_data, 30, 40)
show_percentage(numbers_data, 40, 50)

1 ~ 10 : 17%
10 ~ 20 : 18%
20 ~ 30 : 20%
30 ~ 40 : 26%
40 ~ 50 : 21%


In [290]:
def retrieve_priority_keys(counter, numbers):
    """
    Priority the numbers.
    """
    priority_numbers = []
    most_numbers = retrieve_most_number(counter)
    for v in most_numbers:
        key = v[0]
        values = sorted(v[1], key=lambda x: int(x))
        for i in values:
            if str(i) in numbers:
                priority_numbers.append(f'p{key}')
    return priority_numbers

def retrieve_all_priority_keys(counter, numbers):
    """
    All priority the numbers.
    """
    priority_numbers = []
    for v in numbers:
        priority_numbers.append(retrieve_priority_keys(counter, v))
    return priority_numbers

priority_keys= retrieve_all_priority_keys(counter, numbers_data)
priority_counter = counter_numbers(priority_keys)
priority_counter = sorted(priority_counter.items(), key=lambda x: x[1], reverse=True)

priority_keys_count = {}
for c in priority_keys:
    c.sort()
    key = "-".join(c)
    if key in priority_keys_count:
        priority_keys_count[key] += 1
    else:
        priority_keys_count[key] = 1
print(priority_counter)

for i in priority_keys_count:
    print(i, priority_keys_count[i])

[('p5', 50), ('p4', 32), ('p2', 28), ('p6', 24), ('p3', 21), ('p7', 14), ('p8', 8), ('p1', 3)]
p2-p4-p5-p6-p7-p8 1
p3-p4-p5-p5-p6-p7 1
p2-p4-p5-p5-p5-p7 1
p3-p4-p4-p5-p6-p8 1
p2-p2-p3-p4-p5-p5 1
p3-p4-p5-p5-p6-p8 2
p2-p3-p4-p5-p5-p7 2
p3-p4-p5-p5-p6-p6 1
p2-p3-p4-p5-p5-p5 2
p2-p2-p3-p5-p5-p6 1
p2-p2-p4-p4-p4-p6 1
p2-p3-p3-p5-p7-p8 1
p2-p4-p5-p5-p6-p7 1
p2-p2-p2-p5-p5-p8 1
p1-p3-p4-p4-p5-p6 1
p2-p3-p3-p3-p5-p7 1
p1-p2-p5-p5-p6-p6 1
p2-p3-p4-p4-p5-p6 1
p4-p5-p5-p5-p6-p7 1
p3-p4-p4-p5-p5-p7 1
p2-p3-p4-p6-p7-p8 1
p4-p4-p5-p5-p6-p8 1
p2-p2-p3-p4-p5-p6 1
p2-p2-p6-p6-p6-p7 1
p2-p5-p5-p6-p6-p7 1
p4-p4-p5-p5-p5-p7 1
p1-p2-p2-p4-p4-p6 1


In [291]:
import itertools as it

def combination_numbers(numbers, num_of_combo):
    """
    Combination the numbers.
    """
    combinations = []
    for i in numbers:
        combinations.append(list(it.combinations(i, num_of_combo)))
    return combinations

def all_combination_numbers(numbers, num_of_combo):
    """
    All combination the numbers.
    """
    all_combinations = []
    combinations = combination_numbers(numbers, num_of_combo)
    for i, c in enumerate(combinations):
        # print(i, c)
        for a in c:
            all_combinations.append(a)
    return all_combinations

def counter_combination_numbers(numbers, num_of_combo):
    """
    Count the number of times each combination appears in the last 30 draws.
    """
    combinations = all_combination_numbers(numbers, num_of_combo)
    counter = collections.Counter(combinations)
    return counter

def show_counter_combination_numbers(numbers, num_of_combo):
    """
    Show the number of times each combination appears in the last 30 draws.
    """
    counter = counter_combination_numbers(numbers, num_of_combo)
    counter = sorted(counter.items(), key=lambda x: x[1], reverse=True)
    for k in counter:
        print(k[1], k[0])

def show_numbers_with_combination(numbers, num_of_combo):
    """
    Show the numbers with combination.
    """
    combinations = combination_numbers(numbers, num_of_combo)
    for index, value in enumerate(numbers):
        print("index", index, "value", value)
        for i, c in enumerate(combinations):
            # print(i, c)
            # skip the combination same of slot with numbers
            if index == i: continue
            for x in c:
                # print(x)
                # result = any(a in x for a in value)
                # if result: print(i, x)
                result = [a for a in value if a in x]
                if len(result) == num_of_combo: print(i, x)

show_numbers_with_combination(numbers_data, 2)
# show_counter_combination_numbers(numbers_data, 2)


index 0 value ['9', '30', '31', '32', '37', '40']
19 ('9', '30')
19 ('9', '32')
19 ('9', '40')
19 ('30', '32')
19 ('30', '40')
19 ('32', '40')
22 ('30', '37')
29 ('32', '40')
index 1 value ['19', '21', '23', '26', '37', '39']
2 ('19', '26')
2 ('19', '37')
2 ('26', '37')
6 ('26', '37')
15 ('19', '39')
20 ('19', '37')
21 ('23', '39')
22 ('21', '37')
27 ('21', '37')
index 2 value ['1', '13', '19', '26', '29', '37']
1 ('19', '26')
1 ('19', '37')
1 ('26', '37')
6 ('26', '37')
10 ('1', '29')
20 ('19', '37')
28 ('26', '29')
index 3 value ['2', '15', '20', '30', '47', '48']
7 ('47', '48')
9 ('2', '48')
16 ('2', '15')
22 ('30', '47')
23 ('20', '30')
24 ('20', '48')
28 ('15', '20')
index 4 value ['9', '17', '24', '27', '33', '39']
6 ('17', '24')
index 5 value ['7', '16', '21', '30', '34', '38']
7 ('16', '21')
12 ('30', '38')
14 ('30', '34')
18 ('7', '16')
19 ('30', '34')
22 ('21', '30')
27 ('21', '38')
index 6 value ['11', '17', '24', '26', '37', '41']
1 ('26', '37')
2 ('26', '37')
4 ('17', '24'

In [292]:
import random

def random_numbers_by_priority_keys(most_numbers, priority_list):
    """
    Random the numbers.
    """
    random_numbers = []
    random_slot = {}
    max = len(priority_list)
    while len(random_numbers) < max:
        for i in priority_list:
            number = random.choice(most_numbers[i])
            number_slot = random_slot.get(i) or []
            if number not in number_slot and number not in random_numbers:
                random_numbers.append(number)
                random_slot.update({i: number_slot.append(number)})
                break
    return random_numbers

random_numbers = random_numbers_by_priority_keys(all_most_numbers, ['p2', 'p3', 'p3', 'p5', 'p5', 'p6'])
print(random_numbers)

['31', '33', '25', '45', '36', '4']


In [293]:

def random_numbers_by_priority_keys_exclude(most_numbers, exclude_numbers, priority_list):
    """
    Random the numbers.
    """
    random_numbers = []
    random_slot = {}
    max = len(priority_list)
    while len(random_numbers) < max:
        for i in priority_list:
            number = random.choice(most_numbers[i])
            number_slot = random_slot.get(i) or []
            if number not in number_slot and number not in random_numbers and number not in exclude_numbers:
                random_numbers.append(number)
                random_slot.update({i: number_slot.append(number)})
                break
    return random_numbers

random_numbers_exclude = random_numbers_by_priority_keys_exclude(all_most_numbers, random_numbers, ['p2', 'p3', 'p4', 'p5', 'p5', 'p7'])
print(random_numbers_exclude)

['10', '2', '16', '18', '20', '42']


In [294]:
def random_numbers(most_numbers):
    """
    Random the numbers.
    """
    random_numbers = []
    while len(random_numbers) < 6:
        number = random.choice(most_numbers)
        if number not in random_numbers:
            random_numbers.append(number)
    return random_numbers

def get_priority_by_numbers(numbers, most_numbers):
    """
    Get the priority by numbers.
    """
    priority_keys = []
    for n in numbers:
        for i in most_numbers:
            if n in most_numbers[i]:
                priority_keys.append(i)
    priority_keys.sort()
    return priority_keys

def count_exists(list, by_list):
    count = 0
    for k in list:
        if k in by_list:
            count += 1
    return count

def random_combination(most_numbers, priority_keys, include_keys=[], exclude_keys=[], exclude_numbers=[]):
    """
    Random the new combination.
    """
    numbers = []
    all_numbers = []
    # merge all most numbers to list
    for i in most_numbers:
        all_numbers += most_numbers[i]

    # print(priority_keys)
    # print(exclude_keys)
    while len(numbers) < 6:
        num = random_numbers(all_numbers)
        keys = get_priority_by_numbers(num, most_numbers)
        # print('Failed', keys)
        if keys not in priority_keys and count_exists(keys, exclude_keys) == 0:
            count_include_keys = count_exists(keys, include_keys) if len(include_keys) > 0 else 0
            count_exclude_num = count_exists(num, exclude_numbers) if len(exclude_numbers) > 0 else 0
            if count_include_keys == len(include_keys) and count_exclude_num == 0:
                print('Success', keys)
                numbers = num
    return numbers

# random_combination(all_most_numbers, priority_keys, include_keys=['p5', 'p5'], exclude_keys=['p1'])
random_combination(all_most_numbers, priority_keys, include_keys=['p5'], exclude_keys=['p1'], exclude_numbers=numbers_data[0] + ['46', '19', '36', '29', '8', '44'])

Success ['p2', 'p2', 'p3', 'p4', 'p4', 'p5']


['41', '10', '7', '12', '38', '33']